In [ ]:
!pip install -q pyomo

In [ ]:
from pyomo.environ import *
import pandas as pd
import numpy as np

In [ ]:
!apt-get install -y -qq coinor-cbc

***Question 1*** Write a mathematical model to solve the assignment problem explained above. Define all
the variables and constraints clearly. Use appropriate notations and define appropriate sets
to be used in your optimization problem.

***Solution 1***

Minimize $\sum_{i=1}^{i=n} \sum_{j=1}^{j=n} \sum_{k=1}^{k=n} \sum_{l=1}^{l=n} c_{k,l} d_{i,j} x_{i,k} x_{j,l}$

the varables are $x_{i,j}$ = {0,1} $\forall$ i,j = {1,2,...,n}

Explaination : After optimization factory i will go to position k
(factory) , similary j to position L(factory) So quantity that is travelled between them is $d_{i,j}$ and the cost will be respective to new position  so it will be $c_{k,l}$ so we need to minimize $\sum_{i=1}^{i=n} \sum_{j=1}^{j=n} \sum_{k=1}^{k=n} \sum_{l=1}^{l=n} c_{k,l} d_{i,j} x_{i,k} x_{j,l}$


Here the value of n = 8

Constraints:

* Each factory need to be assigned one location and this can modelled as 

For each **i** , $\sum_{j=1}^{j=n}x_{i,j}$=1 


* Each Location need to be assigned one factory and this can modelled as 

For each **j** , $\sum_{i=1}^{i=n}x_{i,j}$=1


The above equations are equal to 1 because as the variables take 0,1 values so only one variable will take value as 1 and rest zero  so x_{i,j} = 1 indicates that factory i is assigned to location j


Since this is a quadratic assignment problem if we want to solve using cbc solver we need to convert into to linear.So we use Adams and Johnson's linearization.

So we define $y_{i,j,k,l}  = x_{i,j}*x_{k,l} $

So Objective function will be $\sum_{i=1}^{i=n} \sum_{j=1}^{j=n} \sum_{k=1}^{k=n} \sum_{l=1}^{l=n} c_{k,l} d_{i,j} y_{i,j,k,l}$

The constraints :

* $\sum_{i=1}^{i=n} y_{i,j,k,l} = x_{k,l}$$  \forall$ j,k,l = {1,2,...,n} --(1)

* $\sum_{j=1}^{j=n} y_{i,j,k,l} = x_{k,l}$$  \forall$ i,k,l = {1,2,...,n} --(2)

* $y_{i,j,k,l} = y_{k,l,i,j} \forall$ i,j,k,l = {1,2,...,n} --(3)

* $y_{i,j,k,l} >= 0$

* $\sum_{j=1}^{j=n}x_{i,j}$=1 $\forall  i = {1,2,3,...,n}$ --(4)

* $\sum_{i=1}^{i=n}x_{i,j}$=1 $\forall j =  {1,2,3,...,n}$ --(5)


***Question 2***
If there are n factories and n locations, how many variables and constraints are there in
your model?

***Solution 2*** 

Variables:
* So  $Y_{i,j,k,l}$ and i,j,k,l each take values from 1 to n so no of variables is $n^4$ 
* for each $X_{i,j}$ and i,j each take values from 1 to n so no of variables is $n^2$
* Total No of variables = $n^4+n^2$

constraints: 

* each factory should be assigned one location ( so n ) as n factories
* each location should be assigned one factory (so n ) as n locations
* linearyity constraints = for equations (1) & (2) - $ 2* n^{3}$ , (3) - $n^{4}$ , for (4),(5) - $2*n$
* So total no of constraints = $ n^{4} + 2 * n ^{3} + 2* n $

In [21]:
data_q = np.loadtxt("lab7_ex1_q.txt",dtype = "int")

In [22]:
data_q

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 1,  0, 18, 22, 19, 23, 20, 18, 19,  0, 20, 24, 35],
       [ 2, 21,  0, 20,  0, 19,  0, 22, 20, 18, 19,  0, 33],
       [ 3, 22, 23,  0,  0,  0, 17, 16, 24, 16, 18, 24,  0],
       [ 4, 17, 25,  0,  0, 21, 22, 20, 17, 15, 16, 24, 31],
       [ 5, 12, 19, 18, 19,  0, 21, 21, 23, 21,  0,  0, 21],
       [ 6, 20,  0,  0, 17, 21,  0, 20,  0, 19, 17, 22, 20],
       [ 7, 22, 24, 28, 16, 23,  0,  0, 19, 24, 25, 30, 19],
       [ 8, 23, 29, 20, 17, 24, 24, 23,  0, 19, 22, 30, 26],
       [ 9,  0, 28, 29, 21,  0, 24, 18, 18,  0,  0, 22, 23],
       [10, 31, 20,  0, 19, 23,  0, 20, 24, 19,  0, 20,  0],
       [11, 17,  0, 25, 23, 20, 18, 16, 19,  0, 39,  0, 34],
       [12,  0, 32, 24, 26,  0, 19,  0, 18, 37, 21, 20,  0]])

In [23]:
data_c = np.loadtxt("lab7_ex1_c.txt",dtype = "int")

In [24]:
data_c

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
       [ 1,  0, 11, 13,  9, 19, 20, 27, 13, 19, 11, 19, 10],
       [ 2, 11,  0,  8,  9, 11, 19, 10, 15, 12, 23, 24, 11],
       [ 3, 13,  8,  0, 18, 19, 19, 27, 27, 19, 24, 12, 17],
       [ 4,  9,  9, 18,  0, 19, 20, 10, 20, 21, 20, 27, 10],
       [ 5, 19, 11, 25, 19,  0, 21, 17, 26, 20, 14, 24, 17],
       [ 6, 20, 13, 20, 20, 21,  0, 28, 14, 22, 17, 22, 23],
       [ 7, 27, 10, 17, 10, 17, 28,  0, 12, 18, 26, 10, 12],
       [ 8, 13, 15, 27, 20, 19, 14, 12,  0, 27, 10, 19, 17],
       [ 9, 29, 12, 19, 21, 20, 22, 18, 27,  0, 20, 22, 16],
       [10, 11, 20, 24, 10, 14, 17, 26, 10, 10,  0, 25, 21],
       [11, 18, 19, 12, 14, 14, 22, 10, 19, 12, 25,  0, 21],
       [12, 20, 21, 17, 10, 17, 20, 22, 17, 16, 21, 21,  0]])

In [ ]:
n = 8
model = ConcreteModel()
model.x = Var(range(n),range(n),domain = Binary)
model.y = Var(range(n),range(n),range(n),range(n),domain = NonNegativeReals)

In [ ]:
model.objective = Objective(expr = sum(data_q[i+1][k+1]*data_c[j+1][l+1]*model.y[i,j,k,l] for i in range(n) for j in range(n) for k in range(n) for l in range(n)),sense=minimize)

In [ ]:
model.constraints = ConstraintList()

In [ ]:
for i in range(n):
  model.constraints.add(expr = sum(model.x[i,j] for j in range(n))==1)
  model.constraints.add(expr = sum(model.x[j,i] for j in range(n))==1)

In [ ]:
for i in range(n):
  for j in range(n):
    for k in range(n):
      for l in range(n):
        model.constraints.add(expr = model.y[i,j,k,l]==model.y[k,l,i,j])

In [ ]:
for j in range(n):
  for k in range(n):
    for l in range(n):
      model.constraints.add(expr = sum(model.y[i,j,k,l] for i in range(n))==model.x[k,l])
for i in range(n):
  for k in range(n):
    for l in range(n):
      model.constraints.add(expr = sum(model.y[i,j,k,l] for j in range(n))==model.x[k,l])

In [ ]:
model.pprint()

Streaming output truncated to the last 5000 lines.
         139 :   0.0 :                                                                                        y[0,1,7,2] - y[7,2,0,1] :   0.0 :   True
         140 :   0.0 :                                                                                        y[0,1,7,3] - y[7,3,0,1] :   0.0 :   True
         141 :   0.0 :                                                                                        y[0,1,7,4] - y[7,4,0,1] :   0.0 :   True
         142 :   0.0 :                                                                                        y[0,1,7,5] - y[7,5,0,1] :   0.0 :   True
         143 :   0.0 :                                                                                        y[0,1,7,6] - y[7,6,0,1] :   0.0 :   True
         144 :   0.0 :                                                                                        y[0,1,7,7] - y[7,7,0,1] :   0.0 :   True
         145 :   0.0 :                     

In [ ]:
opt_cbc = SolverFactory('cbc')
result = opt_cbc.solve(model)
print("Solver Status: ",result.solver.status)
print("Solver Termination condition: ",result.solver.Termination_condition)
print("Time taken: ",result.solver.time)

Solver Status:  ok
Solver Termination condition:  optimal
Time taken:  631.0199859142303


In [ ]:
print("Objective value :",model.objective())

Objective value : 14889.0


In [ ]:
for i in range(n):
    for j in range(n):
      if model.x[i,j].value ==1:
        print(f" X[{i+1},{j+1}] = 1 Source {i+1} is assigned to destination {j+1}")

 X[1,2] = 1 Source 1 is assigned to destination 2
 X[2,8] = 1 Source 2 is assigned to destination 8
 X[3,7] = 1 Source 3 is assigned to destination 7
 X[4,6] = 1 Source 4 is assigned to destination 6
 X[5,1] = 1 Source 5 is assigned to destination 1
 X[6,3] = 1 Source 6 is assigned to destination 3
 X[7,5] = 1 Source 7 is assigned to destination 5
 X[8,4] = 1 Source 8 is assigned to destination 4


***Question 9***Solve the problem and report which facility must be opened at each location. Source(factory),destination (factories position)

***Solution 9***

* X[1,2] = 1 Source 1 is assigned to destination 2
* X[2,8] = 1 Source 2 is assigned to destination 8
* X[3,7] = 1 Source 3 is assigned to destination 7
* X[4,6] = 1 Source 4 is assigned to destination 6
* X[5,1] = 1 Source 5 is assigned to destination 1
* X[6,3] = 1 Source 6 is assigned to destination 3
* X[7,5] = 1 Source 7 is assigned to destination 5
* X[8,4] = 1 Source 8 is assigned to destination 4



In [ ]:
model.x.domain = NonNegativeReals

In [ ]:
result = opt_cbc.solve(model)
print("Solver Status: ",result.solver.status)
print("Solver Termination condition: ",result.solver.Termination_condition)
print("Objective value :",model.objective())

Solver Status:  ok
Solver Termination condition:  optimal
Objective value : 11790.875009995238


In [ ]:
for i in range(n):
    for j in range(n):
      if model.x[i,j].value != 0:
        print(f" *  X[{i+1},{j+1}] = ",model.x[i,j].value)

 *  X[1,1] =  0.084983499
 *  X[1,2] =  0.33367696
 *  X[1,3] =  0.048316653
 *  X[1,4] =  0.10039981
 *  X[1,5] =  0.051444116
 *  X[1,6] =  0.25047975
 *  X[1,7] =  0.066562267
 *  X[1,8] =  0.064136939
 *  X[2,1] =  0.19073548
 *  X[2,2] =  0.026261001
 *  X[2,3] =  0.11864323
 *  X[2,4] =  0.1917243
 *  X[2,5] =  0.17399296
 *  X[2,6] =  0.022830959
 *  X[2,7] =  0.15180168
 *  X[2,8] =  0.1240104
 *  X[3,1] =  0.10388789
 *  X[3,3] =  0.17341885
 *  X[3,4] =  0.20991576
 *  X[3,5] =  0.15765049
 *  X[3,6] =  0.12777006
 *  X[3,7] =  0.12970289
 *  X[3,8] =  0.09765406
 *  X[4,1] =  0.07739104
 *  X[4,2] =  0.0017389244
 *  X[4,3] =  0.16883651
 *  X[4,4] =  0.041916945
 *  X[4,5] =  0.22827876
 *  X[4,6] =  0.13528148
 *  X[4,7] =  0.11864323
 *  X[4,8] =  0.22791312
 *  X[5,1] =  0.20537663
 *  X[5,2] =  0.14760491
 *  X[5,3] =  0.15701317
 *  X[5,4] =  0.043650923
 *  X[5,5] =  0.038651246
 *  X[5,6] =  0.0772711
 *  X[5,7] =  0.15673067
 *  X[5,8] =  0.17370134
 *  X[6,1] =  0.

***Question 11*** Now change the integer variables in your model to continuous variables, and re-solve the
problem. Report the solution (only the non-zero values of the solution).

***SOlution 11*** 

* Objective value : 11790.875009995238

 *  X[1,1] =  0.084983499
 *  X[1,2] =  0.33367696
 *  X[1,3] =  0.048316653
 *  X[1,4] =  0.10039981
 *  X[1,5] =  0.051444116
 *  X[1,6] =  0.25047975
 *  X[1,7] =  0.066562267
 *  X[1,8] =  0.064136939
 *  X[2,1] =  0.19073548
 *  X[2,2] =  0.026261001
 *  X[2,3] =  0.11864323
 *  X[2,4] =  0.1917243
 *  X[2,5] =  0.17399296
 *  X[2,6] =  0.022830959
 *  X[2,7] =  0.15180168
 *  X[2,8] =  0.1240104
 *  X[3,1] =  0.10388789
 *  X[3,3] =  0.17341885
 *  X[3,4] =  0.20991576
 *  X[3,5] =  0.15765049
 *  X[3,6] =  0.12777006
 *  X[3,7] =  0.12970289
 *  X[3,8] =  0.09765406
 *  X[4,1] =  0.07739104
 *  X[4,2] =  0.0017389244
 *  X[4,3] =  0.16883651
 *  X[4,4] =  0.041916945
 *  X[4,5] =  0.22827876
 *  X[4,6] =  0.13528148
 *  X[4,7] =  0.11864323
 *  X[4,8] =  0.22791312
 *  X[5,1] =  0.20537663
 *  X[5,2] =  0.14760491
 *  X[5,3] =  0.15701317
 *  X[5,4] =  0.043650923
 *  X[5,5] =  0.038651246
 *  X[5,6] =  0.0772711
 *  X[5,7] =  0.15673067
 *  X[5,8] =  0.17370134
 *  X[6,1] =  0.022830959
 *  X[6,3] =  0.19583209
 *  X[6,4] =  0.019085987
 *  X[6,5] =  0.26693
 *  X[6,6] =  0.32892201
 *  X[6,7] =  0.06176476
 *  X[6,8] =  0.1046342
 *  X[7,1] =  0.063140624
 *  X[7,2] =  0.31854682
 *  X[7,3] =  0.06176476
 *  X[7,4] =  0.26544384
 *  X[7,5] =  0.083052425
 *  X[7,6] =  0.023820054
 *  X[7,7] =  0.063140624
 *  X[7,8] =  0.12109086
 *  X[8,1] =  0.25165388
 *  X[8,2] =  0.17217138
 *  X[8,3] =  0.076174743
 *  X[8,4] =  0.12786244
 *  X[8,5] =  3.2418512e-14
 *  X[8,6] =  0.033624599
 *  X[8,7] =  0.25165388
 *  X[8,8] =  0.086859086







***Question 12*** 
 Are the optimal costs for both problems same? Are the values of the variables still
integer-valued? If yes, explain why.

***Solution 12***

The optimal cost for both problems are not same . The values of the variables are not integer valued